# 日本語テキストデータの準備

In [ ]:
from azureml.core import Workspace, Datastore, Dataset
import pandas as pd

Livedoor ニュースのコーパスをダウンロードして利用します。

In [ ]:
from urllib.request import urlretrieve
import tarfile

text_url = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
file_path = "../data/raw/ldcc-20140209.tar.gz"
urlretrieve(text_url, file_path)

In [ ]:
# gz ファイルを解凍します。そこそこ時間がかかります。
with tarfile.open('../data/raw/ldcc-20140209.tar.gz', 'r:gz') as tar:
    tar.extractall(path='../data/processed/livedoor')
    tar.close()

Pandas DataFrame へのロード

In [ ]:
columns = ['url', 'date', 'label', 'title', 'text']
df = pd.DataFrame(columns = columns)
#df.set_index('url',inplace=True)

In [ ]:
path = "../data/processed/livedoor/text"


In [ ]:
for folder_name in os.listdir(path):
    print(folder_name)
    if folder_name.endswith(".txt") :
        continue
    for file in os.listdir(os.path.join(path, folder_name)):
        if folder_name == "LICENSE.txt" :
            continue
        with open(os.path.join(path, folder_name, file), 'r') as f:
            lines = f.read().split('\n')
            if len(lines) == 1:
                continue
            url = lines[0]
            date = lines[1]
            label = folder_name
            title = lines[3]
            text = "".join(lines[4:])
            data = {'url': url, 'date':date, 'label': label, 'title':title, 'text':text}
        s = pd.Series(data)        
        df = df.append(s, ignore_index=True)


In [ ]:
df.to_csv("../data/processed/livedoor-corpus.csv", index=False)


# Azure Machine Learning Dataset 登録

In [ ]:
# Azure ML Workspace への接続
ws = Workspace.from_config()
print(ws)

In [ ]:
# Azure ML Workspace のデフォルトの Datastore を利用
datastore = ws.get_default_datastore()
datastore.upload_files(files=['../data/processed/livedoor-corpus.csv'],
                       target_path='livedoor-corpus',
                       overwrite=True,
                       show_progress=False)

In [ ]:
datastore_path = [(datastore, 'livedoor-corpus/livedoor-corpus.csv')]


In [ ]:
livedoor_ds = Dataset.Tabular.from_delimited_files(path=datastore_path)


In [ ]:
# Dataset の登録 (Tabular 形式)
livedoor_ds.register(workspace=ws, name='livedoor',description='livedoor corpus', create_new_version = True)


In [ ]:
# 登録した Dataset の呼び出し
dataset = Dataset.get_by_name(ws, name='livedoor')
df = dataset.to_pandas_dataframe()

In [ ]:
# 欠損値の除外
df = df[pd.isna(df["text"])==False]
df.head()
